# Исследование поведения пользователей 2018

Получите данные из файла.
Произведите обзор данных и преобразование данных, если оно необходимо.

In [182]:
import pandas as pd
import numpy as np

purchase = pd.read_csv(r'C:\Users\Yana\Documents\purchase.csv', sep=',')
events = pd.read_csv(r'C:\Users\Yana\Documents\7_4_Events.csv', sep=',')

In [183]:
cond2018 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[cond2018]  
lst_reg = registered['user_id'].to_list() # список пользователей, зарег. в 2018
# отфильтровать пользователей зарег в 2018
events_df = events[events.user_id.isin(lst_reg)].copy() 
purchase_df = purchase[purchase.user_id.isin(lst_reg)].copy() 

In [184]:
purchase_df['event_type'] = 'purchase' # добавить ивент тип покупка
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])
# purchase_df = purchase_df.rename(columns={"event_datetime": "start_time"})

events_df = events_df.rename(columns={"id": "event_id"}) # переименовать айди
purchase_df = purchase_df.rename(columns={"id": "purchase_id"}) # переименовать айди

total_events_df = pd.concat([events_df,purchase_df],sort=False) # объединить датафреймы

In [185]:
# изменить тип объект на дата для колонок с датой
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time'])
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


In [186]:
user_levels = total_events_df[~total_events_df['selected_level'].isnull()].copy()
print(user_levels_df['user_id'].nunique()) # 8342 юзеров выбрали уровень сложности
print(user_levels_df.groupby('user_id')['selected_level'].count().mean()) # один уровень на юзера

8342
1.0


Выделите группы пользователей

In [187]:
group_easy = user_levels_df[(user_levels_df['selected_level'] == 'easy')]["user_id"].unique()
group_medium = user_levels_df[(user_levels_df['selected_level'] == 'medium')]["user_id"].unique()
group_hard = user_levels_df[(user_levels_df['selected_level'] == 'hard')]["user_id"].unique()
set_easy = set(group_easy) #список выбравших низкий уровень сложности
set_medium = set(group_medium) #список выбравших средний уровень сложности
set_hard = set(group_hard) #список выбравших высокий уровень сложности
print(len(set_easy)+len(set_medium)+len(set_hard)) # совпадает с ранее рассчитанным количеством

8342


Рассчитайте для каждой группы процент оплат

In [188]:
# сколько пользователей в каждой группе по уровню сложности
groups_df = user_levels_df['selected_level'].value_counts().to_frame().reset_index()
groups_df = groups_df.rename(columns={"selected_level": "users_count"})
groups_df = groups_df.rename(columns={"index": "selected_level"})
groups_df['users_percent'] = (groups_df['users_count']/(user_levels_df['user_id'].nunique())*100).round(decimals=2)

merged_df_1 = user_levels_df.merge(purchase_df, on='user_id', how='outer')
bought_groups_df = merged_df_1.groupby('selected_level')['purchase_id'].count().to_frame().reset_index()
# merged_df_1.pivot_table(values = ['purchase_id'], index = 'selected_level', aggfunc='count')
bought_groups_df = bought_groups_df.rename(columns={"purchase_id": "purchase_count"})
merged_buyers_only = merged_df_1[~merged_df_1['purchase_id'].isnull()].copy()
merged_buyers_only['timedelta'] = merged_buyers_only['event_datetime'] - merged_buyers_only['start_time']

# среднее время между выбором уровня и покупкой в зависимости от уровня
level_buytime_df = merged_buyers_only.groupby('selected_level')['timedelta'].mean().to_frame().reset_index()
level_buytime_df = level_buytime_df.rename(columns={"timedelta": "timedelta_mean_level-to-purchase"})
# merged_buyers_only.groupby('selected_level')['timedelta'].quantile([.25, .5, .75])

level_stats_df = groups_df.merge(bought_groups_df, on='selected_level', how='outer')

# процент оплат для каждой группы
level_stats_df['purchase_percent'] = (level_stats_df['purchase_count'] / level_stats_df['users_count']*100).round(decimals=2)

# полная таблица
full_level_stats_df = level_stats_df.merge(level_buytime_df, on='selected_level',how='outer')
display(full_level_stats_df)

,selected_level,users_count,users_percent,purchase_count,purchase_percent,timedelta_mean_level-to-purchase
0,medium,4645,55.68,969,20.86,3 days 23:14:13.165118679
1,easy,2448,29.35,189,7.72,3 days 14:58:52.941798941
2,hard,1249,14.97,442,35.39,3 days 07:20:41.420814479


**Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты?**

1. наибольшее количество пользователей (4645 человек) выбрало средний уровень сложности, но им в среднем понадобилось больше всего времени (почти 4 дня) чтобы совершить на покупку.

2. наименьшее количество пользователей (1249 человек) выбрало высокий уровень сложности, однако им в среднем понадобилось меньше всего времени (3 дня и 7,35 часа) чтобы совершить покупку.

3. те, кто выбрал низкий уровень сложности (примерно треть от всех, кто выбрал уровень сложности) редко совершали покупку: только 7,72 %.

**Вывод: чем выше уровень трудности выбрал пользователь, тем вероятнее что он совершит покупку.**

Различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности?

In [181]:
purchase_df = purchase_df.rename(columns={"event_datetime": "start_time"})
total_events_2 = pd.concat([events_df,purchase_df],sort=False) # объединить датафреймы

a = total_events_2.pivot_table(
    values='start_time',
    index= ['user_id'],
    columns= 'event_type',
    aggfunc='sum')

a.reset_index(level=0, inplace=True)
del a["tutorial_finish"]
del a["tutorial_start"]
del a["pack_choice"] 
# del a["level_choice"] 
a = a[~a['purchase'].isnull()].copy()
a['registration'] = pd.to_datetime(a['registration'])
a['level_choice'] = pd.to_datetime(a['level_choice'])

easy_df = a[a["user_id"].isin(set_easy)].copy()
easy_df['timedelta'] = easy_df['purchase'] - easy_df['registration'] 
easy_df['timedelta2'] = easy_df['level_choice'] - easy_df['registration'] 
easy_time = easy_df['timedelta'].mean()

medium_df = a[a["user_id"].isin(set_medium)].copy()
medium_df['timedelta'] = medium_df['purchase'] - medium_df['registration'] 
medium_df['timedelta2'] = medium_df['level_choice'] - medium_df['registration'] 
medium_time = medium_df['timedelta'].mean()

hard_df = a[a["user_id"].isin(set_hard)].copy()
hard_df['timedelta'] = hard_df['purchase'] - hard_df['registration'] 
hard_df['timedelta2'] = hard_df['level_choice'] - hard_df['registration'] 
hard_time = hard_df['timedelta'].mean()

print(f'''Временной промежуток между регистрацией и оплатой у групп пользователей с:
1) низким уровнем сложности: {easy_time},
2) средним уровнем сложности: {medium_time},
3) высоким уровнем сложности: {hard_time}''')

#display(easy_df['timedelta2'].mean())
#display(medium_df['timedelta2'].mean())
#display(hard_df['timedelta2'].mean())
#display((easy_df['timedelta2'].mean()+medium_df['timedelta2'].mean()+hard_df['timedelta2'].mean())/3)

Временной промежуток между регистрацией и оплатой у групп пользователей с:
1) низким уровнем сложности: 3 days 22:10:23.211640211,
2) средним уровнем сложности: 4 days 06:12:06.576883384,
3) высоким уровнем сложности: 3 days 14:55:19.257918552


**Существует ли разница во времени между событиями регистрации и оплаты?**
- опять же, быстрее всех совершают покупку пользователи, выбравшие высокий уровень сложности (за 3 дня и 15 часов).
- медленнее всех совершают покупку пользователи, выбравшие средний уровень сложности (за 4 дня и 6,2 часов).
- те, кто выбрал низкий уровень сложности совершают покупку в среднем через 3 дня и 22,2 часов после регистрации.

похоже что время между регистрацией и выбором уровня для разных групп различается мало, и в среднем составляет 7 часов и 15 минут